In [1]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import os
import skimage
import PIL
import matplotlib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imutils import paths

from sklearn import metrics
#from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

C:\Users\Sarah\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# grab the list of images that we'll be describing
# print("[INFO] describing images...")
imagePaths = list(paths.list_images("."+os.sep+"train_vega"))

# initialize the raw pixel intensities matrix, the features matrix, and labels list
rawImages = []
features = []
labels = []
images=[]

In [3]:
#Function for the images treatment
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist(image, [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    
    # handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)

    # otherwise, perform "in place" normalization in OpenCV 3
    else:
        cv2.normalize(hist, hist)
        
    # return the flattened histogram as the feature vector
    return hist.flatten()

def create_distortion(image,number):
    images = []
    for i in range(1,number):
        images.append(np.uint8(image + i*0.5* image.std() * np.random.random(image.shape)))
    return images

def image_treatment(image,label):
    rawImages.append(image_to_feature_vector(image))
    features.append(extract_color_histogram(image))
    labels.append(label)

In [4]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label (assuming that our
    # path as the format: /path/to/dataset/{class}.{image_num}.jpg
    
    image = cv2.imread(imagePath)
    tab_image = create_distortion(image,11)
    
    image_flip = cv2.flip(image,1)
    tab_image_flip = create_distortion(image_flip,11)
   
    images = tab_image + tab_image_flip
    images.append(image)
    images.append(image_flip)
    
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    for im in images:
        image_treatment(im,label)

    # show an update every 10 images
    if i > 0 and i % 10 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))
print("Done")

Done


In [5]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
    rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
    features, labels, test_size=0.25, random_state=42)

In [6]:
svm_kernel = ["linear","poly","rbf","sigmoid"]
# train and evaluate a SVM classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy")
scores_rawpixel = [];
for k in range(len(svm_kernel)) :
    model = SVC(kernel=svm_kernel[k], C=1, random_state=0)
    model.fit(trainRI, trainRL)
    RL_pred = model.predict(testRI)
    acc = model.score(testRI, testRL)
    scores_rawpixel.append(acc)
    print("[INFO] raw pixel accuracy for",svm_kernel[k],"kernel:{:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy
[INFO] raw pixel accuracy for linear kernel:76.00%
[INFO] raw pixel accuracy for poly kernel:72.00%
[INFO] raw pixel accuracy for rbf kernel:56.00%
[INFO] raw pixel accuracy for sigmoid kernel:56.00%


In [8]:
# train and evaluate a SVM classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy")
scores_hist = [];
for k in range(len(svm_kernel)) :
    model = SVC(kernel=svm_kernel[k], C=1, random_state=0)
    model.fit(trainFeat, trainLabels)
    Labels_pred = model.predict(testFeat)
    acc = model.score(testFeat, testLabels)
    scores_hist.append(acc)
    print("[INFO] raw pixel accuracy for",svm_kernel[k],"kernel:{:.2f}%".format(acc * 100))

[INFO] evaluating histogram accuracy
[INFO] raw pixel accuracy for linear kernel:66.00%
[INFO] raw pixel accuracy for poly kernel:56.00%
[INFO] raw pixel accuracy for rbf kernel:56.00%
[INFO] raw pixel accuracy for sigmoid kernel:56.00%


In [10]:
# evaluation modèle SVM
print (metrics.classification_report(testRL, RL_pred))
print ("---------------------------------------------------------------------------")
print (metrics.classification_report(testLabels, Labels_pred))

             precision    recall  f1-score   support

       Ankh       0.00      0.00      0.00        22
     bouche       0.56      1.00      0.72        28

avg / total       0.31      0.56      0.40        50

---------------------------------------------------------------------------
             precision    recall  f1-score   support

       Ankh       0.00      0.00      0.00        22
     bouche       0.56      1.00      0.72        28

avg / total       0.31      0.56      0.40        50



C:\Users\Sarah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
from sklearn import svm
from mlxtend.plotting import plot_decision_regions
import pandas as pd
from numpy import array

X = np.array([testRI, testRL])
y = array(Labels_pred, dtype=int)

# Plot Decision Region using mlxtend's awesome plotting function
plot_decision_regions(X, 
                      y,
                      clf=clf, 
                      legend=2)

# Update plot object with X/Y axis labels and Figure Title
plt.xlabel(X.columns[0], size=14)
plt.ylabel(X.columns[1], size=14)
plt.title('SVM Decision Region Boundary', size=16)

ValueError: invalid literal for int() with base 10: 'bouche'